In [ ]:
import hashlib
import os
import json
import getpass

# File to store user data
USER_DATA_FILE = "users.json"


# Load or initialize user database
def load_users():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as file:
            return json.load(file)
    return {}


def save_users(users):
    with open(USER_DATA_FILE, "w") as file:
        json.dump(users, file)


# Generate random salt
def generate_salt():
    return os.urandom(16)


# Hash password with salt using SHA-1
def hash_password(password, salt):
    return hashlib.sha1(salt + password.encode()).hexdigest()


# Register a new user
def register_user(users):
    username = input("Enter new username: ").strip()
    if username in users:
        print("Username already exists. Try logging in.")
        return

    password = getpass.getpass("Set a password: ")
    if not check_password_complexity(password):
        print(
            "Password too weak. Must be at least 8 characters with upper, lower, digit and symbol."
        )
        return

    salt = generate_salt()
    hashed = hash_password(password, salt)

    users[username] = {"salt": salt.hex(), "hash": hashed, "login_attempts": 0}
    save_users(users)
    print("Registration successful!")


# Check password complexity
def check_password_complexity(pw):
    return (
        len(pw) >= 8
        and any(c.islower() for c in pw)
        and any(c.isupper() for c in pw)
        and any(c.isdigit() for c in pw)
        and any(not c.isalnum() for c in pw)
    )


# Login user
def login_user(users):
    username = input("Username: ").strip()
    if username not in users:
        print("Username not found.")
        return

    user = users[username]
    if user["login_attempts"] >= 3:
        print("Account locked due to too many failed login attempts.")
        return

    password = getpass.getpass("Password: ")
    salt = bytes.fromhex(user["salt"])
    hashed_input = hash_password(password, salt)

    if hashed_input == user["hash"]:
        print("Login successful!")
        user["login_attempts"] = 0
    else:
        user["login_attempts"] += 1
        print(f"Incorrect password. Attempts left: {3 - user['login_attempts']}")

    save_users(users)


# Main menu
def main():
    users = load_users()
    while True:
        print("\nSecure Auth System")
        print("1. Register")
        print("2. Login")
        print("3. Exit")
        choice = input("Select option: ")

        if choice == "1":
            register_user(users)
        elif choice == "2":
            login_user(users)
        elif choice == "3":
            print("Goodbye!")
            break
        else:
            print("Invalid choice.")


if __name__ == "__main__":
    main()



Secure Auth System
1. Register
2. Login
3. Exit


KeyboardInterrupt: Interrupted by user

1. Choice of Hashing Algorithm – SHA-256 (SHA-2 Family)
In our implementation, we chose SHA-256, a member of the SHA-2 family, as the hashing algorithm for password protection. According to the lecture on Cryptography (Lecture 8), SHA-256 was introduced in the revised Secure Hash Standard (FIPS 180-2) and produces a fixed-length 256-bit hash output regardless of input size.
SHA-256 was selected because:
It is currently considered secure and widely adopted in real-world systems.
It provides strong one-way properties, meaning it is infeasible to recover the original password from its hash.
It is resistant to collision attacks, which SHA-1 and MD5 are no longer secure against (as noted in the “Hash Algorithms” slide).
This aligns with Lecture 8’s criteria for secure hash functions:
The output should be deterministic but computationally infeasible to reverse (one-way property).
It should be difficult to find two different messages with the same hash (collision resistance).

2. Implementation of Salting and Its Importance
To further strengthen password storage, we introduced salting—a process where a random, unique string (salt) is added to each password before hashing.
From the lecture on Authentication (Lecture 9), we understand that salting is vital because:
It prevents attackers from using precomputed dictionary or rainbow table attacks, which rely on matching known password hashes.
Even if two users choose the same password, the resulting hashes will be different due to the unique salt, making large-scale hash matching infeasible.
In our approach:
A cryptographically secure random salt is generated for each password using system randomness.
The salt is concatenated with the password and the combination is hashed using SHA-256.
Both the salt and the resulting hash are stored securely in the user database.
This process is in line with what Lecture 9 explains under “Password Salting”: salting makes each password hash user-specific, and storing the salt does not reduce the security of the hash—it simply ensures uniqueness.

3. Password Verification Process
During login, the system:
Retrieves the stored salt and hash associated with the user.
Applies the same hashing process to the password entered by the user, using the stored salt.
Compares the resulting hash with the stored one.
If the hashes match, the user is authenticated successfully. If not, the login attempt fails.
As stated in Lecture 8 under “Message Authentication Code (MAC)” and “Hash Functions”, this approach follows the principle of verifying data integrity by checking that the computed hash matches the original. Even though we do not use a MAC specifically, the process of comparing cryptographic hashes ensures that the password has not been altered and originates from the user.

    "gary30": { //30Gary$$
        "salt": "742006056b6de4bfac8d1a28e1f3b25d",
        "hash": "b890eb3503ddebb7d1f4398008247c325d3e5f9400f4c7515278f750f525db21",
        "login_attempts": 0
    },
    "ishowspeed": { //@Abc1234
        "salt": "6fbfca7a3b38bc8d1d64cb5106f6fdd4",
        "hash": "bd9b9cd17216fc5c93a532463841a537d7d94c5a32400685d699fc99aab74d0d",
        "login_attempts": 0
    }

In [ ]:
import json
import hashlib
import getpass
import random
import os

USER_DATA_FILE = "users.json"


def load_users():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as f:
            return json.load(f)
    return {}


def save_users(users):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(users, f)


def hash_password(password, salt):
    return hashlib.sha256(salt + password.encode()).hexdigest()


def generate_otp():
    return str(random.randint(100000, 999999))


def mfa_login():
    users = load_users()

    while True:
        print("\nMulti-Factor Authentication Login")
        username = input("Enter your username (or type 'exit' to quit): ").strip()
        if username.lower() == "exit":
            print("Exiting MFA system.")
            break

        if username not in users:
            print("Username not found.")
            continue

        user = users[username]
        if user["login_attempts"] >= 3:
            print("Account locked due to too many failed attempts.")
            continue

        # password = getpass.getpass("Enter your password: ")
        password = input("Enter your password: ")

        salt = bytes.fromhex(user["salt"])
        hashed_input = hash_password(password, salt)

        if hashed_input == user["hash"]:
            print("Password correct.")

            # Simulate OTP delivery
            otp = generate_otp()
            print(
                f"OTP sent to {user.get('email', 'your registered email')} (simulated): {otp}"
            )

            entered_otp = input("Enter the OTP: ").strip()
            if entered_otp == otp:
                print("Multi-Factor Authentication Successful. Access granted.")
                user["login_attempts"] = 0
            else:
                print("Incorrect OTP. Access denied.")
                user["login_attempts"] += 1
        else:
            print("Incorrect password.")
            user["login_attempts"] += 1

        save_users(users)


if __name__ == "__main__":
    print("Multi-Factor Authentication Login")
    mfa_login()


1. System Design Overview
Factor 1: Something the user knows (password)
The user inputs a secret password stored in hashed and salted format.
This corresponds to the classic knowledge-based authentication described in Lecture 9, where a password or PIN represents something the user remembers.
Factor 2: Something the user has (OTP)
Upon successful password verification, the system generates a one-time password (OTP) and simulates sending it to the user (e.g., via email).
This represents possession-based authentication, a factor that relies on the user holding a device or token to receive the OTP.
The OTP is randomly generated and changes every login attempt, making it resistant to replay attacks.
2. Security Benefits of MFA Over Single-Factor Authentication
Stronger Authentication Assurance:
 - MFA requires two independent pieces of evidence: knowing the correct password and having access to the OTP.
 - According to Lecture 9 (Slide: Multi-Factor Authentication), combining “something you know” and “something you have” increases confidence that the authentic user is logging in.
Mitigates Password-Based Attacks:
 - Even if a password is compromised through phishing, guessing, or keylogging, an attacker cannot access the system without the second factor.
 - This defends against dictionary attacks and password reuse, which are common threats discussed in Lecture 9 (Slides: Guessing Passwords, Phishing, Spoofing).
Prevents Replay and Man-in-the-Middle Attacks:
 - Because the OTP changes each time, reused or intercepted passwords are ineffective without the new OTP.
 - This limits damage from credential stuffing or spoofing attacks as shown in Lecture 9 (Slides: Spoofing Attacks, Phishing).
Reduces Impact of Stolen Credentials:
 - MFA ensures that even if a password file is leaked or cracked, access is still denied unless the attacker also gains access to the OTP mechanism.
 - This is aligned with Lecture 9’s suggestion that passwords alone are insufficient, especially against modern threats (Slide: More on Authentication).

In [ ]:
import json
import hashlib
import time
import os
import itertools
import string

USER_DATA_FILE = "users.json"


# ---------------------------
# Common Functions
# ---------------------------
def load_users():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as f:
            return json.load(f)
    return {}


def hash_password(password, salt):
    return hashlib.sha256(salt + password.encode()).hexdigest()


def get_user_info(username):
    users = load_users()
    return users.get(username, None)


# ---------------------------
# Method 1: Dictionary Attack
# ---------------------------
def dictionary_attack(username, wordlist):
    user = get_user_info(username)
    if not user:
        print("Username not found.")
        return

    salt = bytes.fromhex(user["salt"])
    real_hash = user["hash"]

    print("\nDictionary Attack Started")
    start = time.time()
    for attempt, password in enumerate(wordlist, start=1):
        hashed = hash_password(password, salt)
        if hashed == real_hash:
            print(f"Password found: '{password}' in {attempt} attempts.")
            print(f"Time: {time.time() - start:.2f} seconds")
            return
    print(f"Dictionary attack failed. ({attempt} attempts)")
    print(f"Time: {time.time() - start:.2f} seconds")


# ---------------------------
# Method 2: Brute Force Attack (Time-limited)
# ---------------------------
def brute_force_attack(username, charset, max_time=60, max_length=10):
    user = get_user_info(username)
    if not user:
        print("Username not found.")
        return

    salt = bytes.fromhex(user["salt"])
    real_hash = user["hash"]

    print(f"\nBrute Force Attack Started (≥8 characters, time-limited to {max_time}s)")
    start = time.time()
    attempt = 0

    for length in range(8, max_length + 1):
        for combo in itertools.product(charset, repeat=length):
            attempt += 1
            password = "".join(combo)
            hashed = hash_password(password, salt)

            if hashed == real_hash:
                print(f"Password cracked! Password is '{password}'")
                print(f"Attempts: {attempt}")
                print(f"Time taken: {time.time() - start:.2f} seconds")
                return

            if time.time() - start > max_time:
                print(f"Time limit of {max_time}s reached. Stopping brute force.")
                print(f"Attempts tried: {attempt}")
                return

    print("Brute-force failed (not found).")


# ---------------------------
# Method 3: Offline Hash Access
# ---------------------------
def show_offline_hash(username):
    user = get_user_info(username)
    if not user:
        print("Username not found.")
        return
    print("\nOffline Access to Stored Hash:")
    print(f"Username: {username}")
    print(f"Salt (hex): {user['salt']}")
    print(f"Hashed Password: {user['hash']}")


# ---------------------------
# Dictionary (Slide 9 + extras)
# ---------------------------
common_passwords = [
    "123456",
    "123456789",
    "qwerty",
    "password",
    "12345678",
    "12345",
    "111111",
    "1234567",
    "sunshine",
    "iloveyou",
    "princess",
    "admin",
    "welcome",
    "football",
    "123123",
    "abc123",
    "1234567890",
    "letmein",
    "1234",
    "baseball",
    "password1",
    "monkey",
    "dragon",
    "shadow",
    "superman",
    "trustno1",
    "whatever",
    "000000",
    "1q2w3e4r",
    "master",
    "qwerty123",
    "login",
    "hello",
    "freedom",
    "starwars",
    "555555",
    "lovely",
    "7777777",
    "123qwe",
    "Test@1234",
    "michael",
    "batman",
    "jesus",
    "hottie",
    "ashley",
    "bailey",
    "charlie",
    "donald",
    "flower",
    "mustang",
    "passw0rd",
    "ninja",
]

# ---------------------------
# Main Interface
# ---------------------------
if __name__ == "__main__":
    print("Task 3: Password Cracking Simulation")
    username = input("Enter username to test: ").strip()
    user = get_user_info(username)

    if not user:
        print("User not found in users.json")
        exit()

    while True:
        print("\n Choose attack method:")
        print("1. Dictionary Attack")
        print("2. Brute Force Attack (8-char, 60s limit)")
        print("3. View Stored Hash (Offline Access)")
        print("4. Exit")
        choice = input("Enter choice (1–4): ").strip()

        if choice == "1":
            dictionary_attack(username, common_passwords)
        elif choice == "2":
            charset = string.ascii_lowercase + string.digits
            brute_force_attack(username, charset, max_time=60)
        elif choice == "3":
            show_offline_hash(username)
        elif choice == "4":
            print("Exiting simulation.")
            break
        else:
            print("Invalid choice. Try again.")



Method  -   Strategy   -   Input Used	-   Exhaustive?     -   Practicality

1	Dictionary Attack   -   Known/common passwords   -   ❌ No   -   Fast, but only cracks weak pw

2	Brute Force Attack   -   All char combos   -   ✅ Yes   -   Slow, time-limited in script

3	Offline Access   -   Just views salt+hash   -   ❌ No   -   Info exposure only



All too weak, hashcat is used to crack the password

# Password Cracking with Hashcat - README

This README outlines the steps for using Hashcat to simulate a password cracking attack for Task 3 as the python methods are 
all failed.

---

## 🔐 Password Sample (from `users.json`)
```json
"ishowspeed": {
    "salt": "6fbfca7a3b38bc8d1d64cb5106f6fdd4",
    "hash": "bd9b9cd17216fc5c93a532463841a537d7d94c5a32400685d699fc99aab74d0d"
}
```
This password is hashed using **SHA-256**, with the **salt prepended** to the plaintext password.

Hashcat mode for this: `1410` (SHA256(salt.password))

---

## ✅ Step-by-Step Instructions

### Step 1: Install Hashcat
- **macOS**:
  ```bash
  brew install hashcat
  ```
- **Ubuntu/Debian**:
  ```bash
  sudo apt install hashcat
  ```

### Step 2: Create Hashes Input File
Save this line into a file named `hashes.txt`:
```txt
bd9b9cd17216fc5c93a532463841a537d7d94c5a32400685d699fc99aab74d0d:6fbfca7a3b38bc8d1d64cb5106f6fdd4
```

### Step 3: Use or Expand Your Wordlist

#### Option A: Use Default Wordlist
Create a file `mywords.txt` with candidate passwords:
```txt
123456
123456789
qwerty
password
12345678
12345
111111
1234567
sunshine
iloveyou
princess
admin
welcome
football
123123
abc123
1234567890
letmein
1234
baseball
password1
monkey
dragon
shadow
superman
trustno1
whatever
000000
1q2w3e4r
master
qwerty123
login
hello
freedom
starwars
555555
lovely
7777777
123qwe
Test@1234
michael
batman
jesus
hottie
ashley
bailey
charlie
donald
flower
mustang
passw0rd
ninja
ishowspeed
ishowspeed123
speed123
Speed2024
iShowSpeed!
hello123
speedrun
gaminglife
mynameisishow
ishowfan
```

#### Option B: Expand Wordlist with RockYou.txt (Recommended)
Download and use the famous `rockyou.txt` wordlist (over 14 million entries):
```bash
curl -L -o rockyou.txt https://github.com/brannondorsey/naive-hashcat/releases/download/data/rockyou.txt
```
Then run Hashcat with:
```bash
hashcat -m 1410 -a 0 -o cracked.txt hashes.txt rockyou.txt
```

---

### Step 4: Run Hashcat
Use the appropriate command based on your wordlist:
```bash
# For custom wordlist
hashcat -m 1410 -a 0 -o cracked.txt hashes.txt mywords.txt

# For rockyou.txt
hashcat -m 1410 -a 0 -o cracked.txt hashes.txt rockyou.txt
```

### Step 5: Check Cracked Result
```bash
cat cracked.txt
```
Expected output format:
```
bd9b9cd17216fc5c93a532463841a537d7d94c5a32400685d699fc99aab74d0d:6fbfca7a3b38bc8d1d64cb5106f6fdd4:actual_password
```

---

## 💡 Recommendations
- Always use strong, random passwords.
- Avoid dictionary words.
- Use MFA (Multi-Factor Authentication).
- Consider key stretching or password managers.

---

## 📅 Task 3 Report Guidance
- **Tool Used**: Hashcat v6.x
- **Hash Mode**: 1410
- **Password Found?** Yes / No
- **Time Taken**: _Measured in seconds/minutes_
- **Wordlist Size**: _Number of entries_
- **Security Recommendations**:
  - Enforce strong passwords (length >12, mixed characters).
  - Apply account lockouts or login throttling.
  - Use bcrypt/Argon2 for better protection.

---

## 📂 Files
```
├── hashes.txt         # hash:salt pairs
├── mywords.txt        # small custom wordlist
├── rockyou.txt        # large public wordlist
├── cracked.txt        # output from Hashcat
```



In [3]:
import hashlib

salt = bytes.fromhex("6fbfca7a3b38bc8d1d64cb5106f6fdd4")
target_hash = "bd9b9cd17216fc5c93a532463841a537d7d94c5a32400685d699fc99aab74d0d"

password = '@Abc1234"'

hashed = hashlib.sha256(salt + password.encode()).hexdigest()

print("Match ✅" if hashed == target_hash else "No match ❌")


No match ❌
